In [1]:
#!pip install tensorflow==2.8.0

In [2]:
import cv2 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [3]:
model = load_model('/kaggle/input/arquivos-card-20/Material/modelo_02_expressoes.h5')

In [22]:
video_file = '/kaggle/input/arquivos-card-20/Material/Videos/video_teste04.mp4'
cap = cv2.VideoCapture(video_file) # carrega o vídeo 
conected, video = cap.read() # conected: bool para indicar se o vídeo foi carregado; video: objeto do vídeo
print(conected, video.shape)

True (360, 640, 3)


# Redimensionando o vídeo para acelerar o treinamento

In [23]:
resize = True # enabler do processo
max_width = 600 # a altura é definida depois, proporcionalmente

if (resize and video.shape[1] > max_width):
    proportion = video.shape[1]/video.shape[0] # calcula a proporção das dimensões do video
    video_width = max_width
    video_height = int(video_width/proportion) # define a altura de acordo com a nova largura e a proporção original
else: # caso o resize não for feito
    video_width = video.shape[1]
    video_height = video.shape[0]

# Definindo as configurações do output

In [24]:
file_name = 'resultado_video_teste04.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID') # especificação do codec por fourcc
fps = 24

# criando o objeto de saída final:
video_output = cv2.VideoWriter(file_name, # nome
                               fourcc, # codec
                               fps, # frames por segundo
                               (video_width, video_height)) # dimensões

# Processamento e gravação do resultado

In [25]:
from tensorflow.keras.preprocessing.image import img_to_array
import time

# caminho para o modelo de detecção de faces:
haarcascade_faces = '/kaggle/input/arquivos-card-20/Material/haarcascade_frontalface_default.xml'
small_font, big_font = 0.4, 0.7
font = cv2.FONT_HERSHEY_SIMPLEX
emotions = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']

while (1): # vai rodar até o fim do vídeo
    conected, frame = cap.read() # conected: indica se o frame foi lido; frame: o objeto de frame
    
    if not conected: # se o frame não foi capturado, encerra o processamento
        print('Finishing')
        break
    
    t = time.time() # registro de tempo do início da iteração
    
    if resize:
        frame = cv2.resize(frame, (video_width, video_height))
    
    face_cascade = cv2.CascadeClassifier(haarcascade_faces) # carrega o modelo para detecção de ROIs
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # converte de BGR para escala de cinza, o que deixa a imagem 3x mais leve
    
    # aplica o modelo de detecção de faces:
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))
    
    if len(faces) > 0: # se alguma face foi detectada
        for (x, y, w, h) in faces: # coodernadas da face
            frame = cv2.rectangle(frame, (x,y), (x+w, y+h+10), (255,50,50), 2) # desenha um retângulo em torno da face
            roi = gray[y: y+h, x: x+w] # extrai a ROI
            roi = cv2.resize(roi, (48, 48)) # faz o redimensionamento da ROI de acordo com o modelo
            roi = roi.astype('float')/255.0 # converte para float e normaliza
            roi = img_to_array(roi) # converte para array e adiciona a dimensão unitário de canal de cor
            roi = np.expand_dims(roi, axis = 0) # adiciona a dimensão de batch
            
            preds = model.predict(roi)[0] # faz a predição e retorna as probabilidades para cada emoção
            
            #if preds is not None: # se houve de fato um resultado retornado
            label = np.argmax(preds) # pega o índice (que coincide com a classe) da maior prob
            cv2.putText(frame, emotions[label], (x, y - 10), font, big_font, (255, 255, 255), 1, cv2.LINE_AA) # adiciona o texto, em branco, à face com o nome da emoção de detectada
    
    # adiciona um texto indicando o tempo de processamento da(s) face(s) no frame:
    cv2.putText(frame, f'frame processed in {time.time() - t:.2f} seconds',
                (20, video_height-20), font, small_font, (255, 255, 255),
                1, cv2.LINE_AA)
    
    video_output.write(frame) # grava o frame no arquivo de saída
print("Finished")
video_output.release() # cria o arquivo do video e salva no diretório de trabalho

Finishing
Finished
